In [109]:
import pyDOE2 as ff
import numpy as np
import pandas as pd
from scipy.stats import f as Fisher
from scipy.stats import t as Student

### Авторы ###

In [110]:
Authors = ["Danis", "Myrat", "Vlad","Alia","Misha"]
Author = Authors[4]
Author

'Misha'

In [111]:
factor_count = 2 # Количество факторов
y_count = 3 # Количество параллельных опытов для каждой комбинации факторов
experiment_count = factor_count**2 # Количество экспериментов
name_y = 'y average' # Название столбца для построения регрессии
table = ff.ff2n(factor_count) # Матрица планирования
table = np.array(table, dtype=np.int8)
table

array([[-1, -1],
       [ 1, -1],
       [-1,  1],
       [ 1,  1]], dtype=int8)

In [112]:
database = pd.read_excel('Data_' + Author + '.xlsx')
name_database = database.columns
Pr = np.unique(np.array(database[name_database[0]]))
print(Pr)
Re = np.array(np.float64(name_database[1:]))
print(Re)
x10 = (np.log(Pr[-1]) + np.log(Pr[0]))/2
x20 = (np.log(Re[-1]) + np.log(Re[0]))/2
I1 = np.log(Pr[-1]) - x10
I2 = np.log(Re[-1]) - x20

database = database[database.columns[1:]]
database = database.to_numpy()

data = []
for i in range(y_count):
    data.append(list(zip(np.log(database[y_count*i, :] - 0.43),
                         np.log(database[y_count*i + 1, :] - 0.43),
                         np.log(database[y_count*i + 2, :] - 0.43))))  # Хранение опытов для каждой комбинации факторов
    # data.append(list(zip(database[y_count*i,:],
    #                      database[y_count*i + 1,:],
    #                      database[y_count*i + 2,:]))) # хранение опытов для каждой комбинации факторов
# data


[ 1  6 11]
[0.004  0.0047 0.09  ]


In [113]:
dtypes = np.dtype(
    [   
        ("x0", int),
        ("x1", int),
        ("x2", int),
        ("y1", np.float64),
        ("y2", np.float64),
        ('y3', np.float64),
        ('y average', np.float64)
    ]
)
df = pd.DataFrame(np.zeros(experiment_count, dtype=dtypes))

In [114]:
for i in range(experiment_count):
    idx1 = table[i,0] # x1
    idx2 = table[i,1] # x2
    ys = np.array(data[idx1 + 1][idx2 + 1],dtype=np.float64) # Совокупность значений y
    #print(ys)
    df.iloc[i] = {'x0': 1,'x1': idx1 , 'x2': idx2 , 'y1': ys[0], 'y2': ys[1], 'y3': ys[2], 'y average': np.mean(ys)}
fixed_y = df[name_y] # y1 - столбец целевой переменной для построения регрессии
# df

In [115]:
a = np.zeros(factor_count + 1)
for i,name in enumerate(df.columns[:factor_count + 1]):
    a[i] = (1./(experiment_count)) * np.sum(df[name] * fixed_y)
print(f"Коэффициенты -> a0: {a[0]}\t a1: {a[1]}\t a2: {a[2]}")
regression = lambda x: np.sum(x*a) #a[0]*x[0] + a[1]*x[1] + a[2]*x[2], где x[0]=1 - фиктивный фактор

Коэффициенты -> a0: -0.7457149014152793	 a1: 0.3747672376667529	 a2: 0.21009503438056168


In [116]:
dtypes = np.dtype(
    [
        ('(y1-y_avg)^2', np.float64),
        ('(y2-y_avg)^2', np.float64),
        ('(y3-y_avg)^2', np.float64),
        ("Variance", np.float64),
        ('y regression_' + name_y, np.float64),
        ('Δy^2', np.float64)
    ]
)
df_tmp = pd.DataFrame(np.zeros(experiment_count, dtype=dtypes))
df = pd.concat([df, df_tmp], axis=1)
# df

In [117]:

df['y regression_' + name_y] = [regression(i) for i in df[df.columns[:factor_count + 1]].to_numpy()]
df['Δy^2'] = (fixed_y - df['y regression_' + name_y]) ** 2

tmp = df.columns[factor_count + 1:(factor_count + 1) + y_count]
#print(tmp)
for idx,name in enumerate(df_tmp.columns[:factor_count + 1]):
    df[name] = (df[tmp[idx]] - df['y average'])**2
    df['Variance'] += df[name]
del tmp
# df

### Проверка адекватности модели ###

In [118]:
f = experiment_count - (factor_count + 1)
print(f"f = {f}")

S_ad = (1./f) * df['Δy^2'].sum()
print(f"S_ад = {S_ad}")

S_y = df['Variance'].sum() * (1/(experiment_count * (y_count - 1)))
print(f"S_y = {S_y}")


f = 1
S_ад = 0.00023138125140464646
S_y = 0.0012559327089381486


In [119]:
p = 0.05 # Принятый уровень значимости

#### Критерий Стьюдента ####

In [120]:
S_bj = np.sqrt(np.abs(S_y/experiment_count))
print(f"S_bj = {S_bj}")

t_table = Student.ppf(1-p/2, experiment_count*(y_count-1)) # Табличный критерий Стьюдента
print(f"Табличное значение критерия Стьюдента: {t_table}")

t = np.zeros(factor_count + 1)
for i in range(len(t)):
    t[i] = np.abs(a[i])/S_bj
    if t[i] > t_table:
        print(f"Коэффициент a{i} является значимым! {t[i]}")
    else:
        print(f"Коэффициент a{i} является незначимым!")

S_bj = 0.017719570458522328
Табличное значение критерия Стьюдента: 2.306004135204166
Коэффициент a0 является значимым! 42.08425385710315
Коэффициент a1 является значимым! 21.14990532891312
Коэффициент a2 является значимым! 11.856666326779681


#### Критерий Фишера ####

In [121]:
F_table = Fisher.ppf(1-p, f, experiment_count*(y_count-1)) # Табличный критерий Фишера
print(f"Табличное значение критерия Фишера: {F_table}")

F = S_ad/S_y
print(f"Модель адекватна c F = {F}!") if F < F_table else print(f"Модель неадекватна c F = {F}!")


Табличное значение критерия Фишера: 5.317655071578712
Модель адекватна c F = 0.1842306118456553!


### Интерпретация результатов ###

In [122]:
df

,x0,x1,x2,y1,y2,y3,y average,(y1-y_avg)^2,(y2-y_avg)^2,(y3-y_avg)^2,Variance,y regression_y average,Δy^2
0,1,-1,-1,-1.319010,-1.350155,-1.299750,-1.322972,0.000016,0.000739,0.000539,0.001294,-1.330577,0.000058
1,1,1,-1,-0.597110,-0.620827,-0.548008,-0.588648,0.000072,0.001035,0.001652,0.002759,-0.581043,0.000058
2,1,-1,1,-0.947781,-0.949589,-0.856608,-0.917993,0.000887,0.000998,0.003768,0.005654,-0.910387,0.000058
3,1,1,1,-0.138228,-0.159582,-0.161931,-0.153247,0.000226,0.000040,0.000075,0.000341,-0.160853,0.000058


In [123]:
b1 = a[1]/I1
b2 = a[2]/I2
b0 = a[0] - b1 * x10 - b2 * x20

print(b0,b1,b2)

c = np.exp(b0)
n = b1
m = b2
print(f"c = {c}, n = {n}, m = {m}")
Nu = lambda pr,re: 0.43 + c*(re**m)*(pr**n)

-0.5854184662369694 0.3125801547032497 0.13495680188824535
c = 0.5568727805903, n = 0.3125801547032497, m = 0.13495680188824535


#### Проверка коэффициентов c, m, n для уравнения Нуссельта ####

In [124]:
table_res = np.zeros((3,3))
for i in range(len(Pr)):
    for j in range(len(Re)):
        table_res[i][j] = Nu(Pr[i], Re[j])
table_res = pd.DataFrame(table_res, columns=Re, index=Pr)
table_res

,0.0040,0.0047,0.0900
1,0.694325,0.700141,0.832368
6,0.892778,0.902960,1.134464
11,0.989315,1.001621,1.281418


In [125]:
np.exp(-3.28)

np.float64(0.03762825680717622)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Мурат&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Данис ###
## &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;↓&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ↓ ##

![cat](https://www.meme-arsenal.com/memes/27366f67c1b492100085bfebf3415ca2.jpg)